In [1]:
import pandas as pd
import sys
import requests as r

sys.path.append('../helper_functions')
import github_utils as gh
import tokenlist_utils as tl
import duneapi_utils as dapi

url_content = [
    # owner_name, repo_name, path_name, file_name
      ['ethereum-optimism','ethereum-optimism.github.io','','optimism.tokenlist.json',] #OP Bridge - https://github.com/ethereum-optimism/ethereum-optimism.github.io/blob/master/optimism.tokenlist.json
    , ['rainbow-me','rainbow-token-list','output','lean-rainbow-token-list.json'] #Rainbow - https://github.com/rainbow-me/rainbow-token-list/blob/main/output/lean-rainbow-token-list.json
    , ['velodrome-finance','docs','public','tokenlist.json'] #Velo - https://github.com/velodrome-finance/docs/blob/aa557db47248b88af73dcdebfbc161e2b93da712/public/tokenlist.json#L4
]

In [2]:
dune_list_query_id = 2420477 #https://dune.com/queries/2420477

In [3]:
# Load the JSON data from the URL
df_list = []
for gh_url in url_content:

    owner_name = gh_url[0]
    repo_name = gh_url[1]
    path_name = gh_url[2]
    file_name = gh_url[3]

    gh_file = gh.get_file_url_from_github(owner_name,repo_name,path_name,file_name)
    res = r.get(gh_file)
    data = res.json()
    if owner_name == 'ethereum-optimism':
        op_list = tl.generate_op_table_from_tokenlist(data)
    tmp = tl.generate_table_from_tokenlist(data)
    tmp['list_name'] = owner_name
    df_list.append(tmp)

df = pd.concat(df_list)
df = df[df['chainId'] ==10] #Get OP Version

df = df.groupby(['chainId', 'address', 'name', 'symbol', 'decimals'])['list_name'].agg(list)
df = df.reset_index()
# display(df)

In [4]:
custom_bridge_list = op_list[
                (op_list['chainId'] == 10)
                & (op_list['optimismBridgeAddress'] != '0x4200000000000000000000000000000000000010')
                & (op_list['optimismBridgeAddress'] != '')
                ]

display(custom_bridge_list)

,chainId,address,name,symbol,decimals,optimismBridgeAddress
31,10,0xc98B98d17435AA00830c87eA02474C5007E1f272,BitBTC,BitBTC,18,0x158F513096923fF2d3aab2BcF4478536de6725e2
79,10,0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1,Dai Stablecoin,DAI,18,0x467194771dAe2967Aef3ECbEDD3Bf9a310C76C65
99,10,0xe7BC9b3A936F122f08AAC3b1fac3C3eC29A78874,Eco,ECO,18,0xAa029BbdC947F5205fBa0F3C11b592420B58f824
316,10,0x8700daec35af8ff88c16bdf0418774cb3d7599b4,Synthetix,SNX,18,0x136b1EC699c62b0606854056f02dC7Bb80482d63
391,10,0xbfD291DA8A403DAAF7e5E9DC1ec0aCEaCd4848B9,dForce USD,USX,18,0xc76cbFbAfD41761279E3EDb23Fd831Ccb74D5D67
429,10,0x1F32b1c2345538c0c6f582fCB022739c4A194Ebb,Wrapped liquid staked Ether 2.0,wstETH,18,0x8E01013243a96601a86eb3153F0d9Fa4fbFb6957


In [5]:
dune_list = dapi.get_dune_data(dune_list_query_id)
# display(dune_list)

2023-09-25 15:35:43.505 | INFO     | duneapi_utils:get_dune_data:44 - Results available at https://dune.com/queries/2420477
2023-09-25 15:35:44.235 | INFO     | duneapi_utils:get_dune_data:89 - ✨ Results saved as csv_outputs/my_query_results.csv, with 561 rows and 4 columns.


In [6]:
#token addresses to lower
df['contract_address'] = df['address'].str.lower().str.strip()
dune_list['contract_address'] = dune_list['contract_address'].str.lower().str.strip()

In [7]:
adds = df[~df['contract_address'].isin(dune_list['contract_address'])]
# display(adds[['address','symbol','decimals', 'name','list_name']])


In [8]:

# Assuming your DataFrame is named 'adds'
for index, row in adds.iterrows():
    sql_values_str = f",({row['address']}, '{row['symbol']}', {row['decimals']}, 'underlying')"
    print(sql_values_str)

,(0x03e5b5930f3a42b56af4a982b10e60957bdd2f61, 'D2D', 18, 'underlying')
,(0x629c2fd5d5f432357465b59d7832389a89956f0b, 'COC', 18, 'underlying')
,(0x6c518f9D1a163379235816c543E62922a79863Fa, 'bwAJNA', 18, 'underlying')
,(0x76c37F9949e05b37C8373d155C1Fef46a6858481, 'ePendle', 18, 'underlying')
,(0x8368Dca5CE2a4Db530c0F6e535d90B6826428Dee, 'FPIS', 18, 'underlying')
,(0x8637725aDa78db0674a679CeA2A5e0A0869EF4A1, 'NFTE', 18, 'underlying')
,(0x9046D36440290FfDE54FE0DD84Db8b1CfEE9107B, 'YFI', 18, 'underlying')
,(0x96bBD55479e9923512dcc95Eb7Df5edDe6FB9874, 'wOpenX', 18, 'underlying')
,(0xAd42D013ac31486B73b6b059e748172994736426, '1INCH', 18, 'underlying')
,(0xC52D7F23a2e460248Db6eE192Cb23dD12bDDCbf6, 'crvUSD', 18, 'underlying')
,(0xa925f4057d6E6C8FAf8bdE537Ad14BA91A1D0337, 'SYNTH', 18, 'underlying')
,(0xaf3A6f67Af1624d3878A8d30b09FAe7915DcA2a0, 'EQB', 18, 'underlying')
,(0xc5d43a94e26fca47a9b21cf547ae4aa0268670e1, 'FPI', 18, 'underlying')
,(0xc871cCf95024eFa2CbcE69B5B775D2a1DcF49c1B, 'GROW', 18, 